# Generating some test matrices for variable problem sizes

The dictionary where they are stored has the following keys:

keys = [ "standard", "early_inversion", "late_inversion" ], indicating
 - S, standard: a list of matrices of growing dimensions you can exponentiate as they are
 - E, early_inversion: an equivalent list that you may try to see what happens when "inverting the evolution" i.e.
   e^(-i S[m] t) e^(+i E[m] t) or 1-st order Trotterized e^(-i t (S[m]-E[m])) 
   given m the chosen size of the problem
 - L< late_inversion: same list as E, to test eventual numerical problems when L->S

In [1]:
%matplotlib inline

import qinfer as qi
#import qutip as qt
import numpy as np
import scipy as sp
from IPython.display import display, Math, Latex
from functools import partial
import matplotlib.pyplot as plt
import importlib as imp

In [2]:
import datetime
import os as os
import time as time

In [3]:
import logging as logging
import sys

%run LoggingCode.py

In [4]:
# tell the user where files will be saved
dire = os.getcwd()
dire = dire+"/tests/T"
dire = os.path.normpath(dire)
print('Files will be saved in:')
print(dire)

Files will be saved in:
C:\Users\ag15141\Local Documents\Cloud\Dropbox (Qsim)\QML_share_stateofart\QMD\testing_notebooks\tests\T


In [5]:
# Add personalized library to PYTHONPATH
sys.path.append(os.path.join("..","Libraries","QML_lib"))

In [6]:
from Norms import *
from IOfuncts import *
from EvalLoss import *

import ProbeStates as pros
import multiPGH as mpgh
import GenSim_IQLE as gsi
import Evo as evo

In [7]:
imp.reload(pros)
imp.reload(mpgh)
imp.reload(gsi)
imp.reload(evo)

<module 'Evo' from '..\\Libraries\\QML_lib\\Evo.py'>

## Build up the matrices

In [9]:
allmatrices = []
earlyinvmatr = []
lateinvmatr = []

oplist = [evo.sigmax(), evo.sigmay(), evo.sigmaz()]

for size in range(2,11):
    select=np.round((len(oplist)-1)*np.random.rand(size))
    newoplist = [oplist[int(i)] for i in select]
    params=np.random.rand(size)
    earlyparams=np.array([np.random.normal(loc=param, scale=0.5) for param in params])
    lateparams=np.array([np.random.normal(loc=param, scale=0.001) for param in params])
    
    for i in range(len(params)-1):
        if i==0:
            output = np.tensordot(params[i]*newoplist[i], params[i+1]*newoplist[i+1], axes=0)
            earlyout = np.tensordot(earlyparams[i]*newoplist[i], earlyparams[i+1]*newoplist[i+1], axes=0)
            lateout = np.tensordot(lateparams[i]*newoplist[i], lateparams[i+1]*newoplist[i+1], axes=0)
#             print('Done'+str(i))
        else:
            output = np.tensordot(output, params[i+1]*newoplist[i+1], axes=0)
            earlyout = np.tensordot(earlyout, earlyparams[i+1]*newoplist[i+1], axes=0)
            lateout = np.tensordot(lateout, lateparams[i+1]*newoplist[i+1], axes=0)
#             print('Done'+str(i))
    
    output = np.reshape(output, [2**size,2**size])
    earlyout = np.reshape(earlyout, [2**size,2**size])
    lateout = np.reshape(lateout, [2**size,2**size])
    print('Matrix of size'+str(output.shape))
#     print('Matrix of size'+str(earlyout.shape))
#     print('Matrix of size'+str(lateout.shape))
    allmatrices.append(output)
    earlyinvmatr.append(earlyout)
    lateinvmatr.append(lateout)

Matrix of size(4, 4)
Matrix of size(8, 8)
Matrix of size(16, 16)
Matrix of size(32, 32)
Matrix of size(64, 64)
Matrix of size(128, 128)
Matrix of size(256, 256)
Matrix of size(512, 512)
Matrix of size(1024, 1024)


In [238]:
keys = [ "standard", "early_inversion", "late_inversion" ]
keys

['standard', 'early_inversion', 'late_inversion']

In [250]:
[matrix.shape for matrix in matrixdict["standard"]]

[(4, 4),
 (8, 8),
 (16, 16),
 (32, 32),
 (64, 64),
 (128, 128),
 (256, 256),
 (512, 512),
 (1024, 1024)]

In [239]:
values = [allmatrices, earlyinvmatr, lateinvmatr ]

In [240]:
matrixdict = dict(zip(keys, values))

## Import/Export stuff

In [245]:
import pickle
PIK = "evo_matrices_dims"+str(allmatrices[0].shape[0])+"-"+str(allmatrices[-1].shape[0])+".dat"

In [246]:
with open(PIK, "wb") as f:
    pickle.dump(matrixdict, f)

In [247]:
with open(PIK, "rb") as f:
    dataold = pickle.load(f)